# Training Runs
Below are some of the runs executed to acquire training/testing data for our project.

## Installs

In [ ]:
!pip install tensorflow-model-optimization

In [ ]:
from __future__ import print_function
import keras
import tempfile
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import CSVLogger  #, UpdatePruningStep
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_callbacks
import tensorflow_model_optimization as tfmot
import numpy as np
import time
import os
import pickle
from keras.datasets import cifar100
import tensorflow as tf

2022-12-12 23:11:57.310349: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 23:11:57.482268: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-12 23:11:58.380975: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-12-12 23:11:58.381098: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n' .join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Dec 12 23:12:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    29W /  70W |      0MiB / 15360MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Model Setup

In [ ]:
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [ ]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [ ]:
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:
# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------

In [ ]:
def tflite_conv(model,path,quant=False):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if quant == True:
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
    pruned_tflite_model = converter.convert()
    with open(path, 'wb') as f:
      f.write(pruned_tflite_model)
    print('Saved pruned TFLite model to:',path)

In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
path = os.getcwd()
path

'/home/yt2776/COMS6998/final_proj'

In [ ]:
def resnet_training(x_train,y_train,x_test, y_test,layers,frequency,\
                    initial_sparsity,final_sparsity,gpu="T4",\
                    const=True, poly=False,file_name='cifar10_test1',\
                    num_classes=10,begin_step=0,end_step='default'):

    # Default parameters
    batch_size = 128 
    epochs = 300
    data_augmentation = False #True
    n = layers

    # Computed depth from supplied model parameter n
    depth = n * 6 + 2

    # Input image dimensions.
    input_shape = x_train.shape[1:]

    # Subtracting pixel mean improves accuracy
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print('y_train shape:', y_train.shape)

    ####Changes start#####
    num_images = x_train.shape[0] #* (1 - validation_split)

    if end_step == 'default':
        end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

    #hyperparameters: initial_sparsity=0.50, final_sparsity=0.80
    if poly:
        pruning_params = {
              'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
                                                                       final_sparsity=final_sparsity,
                                                                       begin_step=begin_step,
                                                                       end_step=end_step,
                                                                      frequency=frequency)
        }
    if const:
            pruning_params = {
              'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=final_sparsity,
                                                                       begin_step=begin_step,
                                                                       end_step=end_step,
                                                                       frequency=frequency)
        }

    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
    model = resnet_v1(input_shape=input_shape, depth=depth,num_classes=num_classes)
    model = prune_low_magnitude(model, **pruning_params)    #_for_pruning

    model.compile(loss= 'categorical_crossentropy',     #''  tf.  keras.losses.SparseCategoricalCrossentropy(from_logits=True)
                  optimizer=Adam(lr=lr_schedule(0)),
                  metrics=['accuracy'])
    ####Changes end#####
    model.summary()

    # Prepare model model saving directory.
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)

    model_run=file_name+'_'+str(depth)+'_'+gpu
    model_path=model_run+'.h5'
    print("RRrrrrrR",model_path)
    
    filepath = os.path.join(save_dir, model_path)
    print("RRR",filepath)
    
    print(model_run)

    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(filepath=filepath,
                                 monitor='val_acc',
                                 verbose=1,
                                 save_best_only=True)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                   cooldown=0,
                                   patience=5,
                                   min_lr=0.5e-6)
    logdir = tempfile.mkdtemp()
    time_callback = TimeHistory()
    logname=path + '/log_'+model_run+'.csv'
    csv_logger = CSVLogger(logname, append=True, separator=',')
    callbacks = [checkpoint, lr_reducer, lr_scheduler, csv_logger, time_callback, pruning_callbacks.UpdatePruningStep(), tfmot.sparsity.keras.PruningSummaries(log_dir=logdir)]

    st = time.time()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
    #%tensorboard --logdir={logdir}
    training_time = time.time() - st

    print(f"overall training time is {training_time}")
    epoch_times = time_callback.times
    print(f"each epoch training time is {epoch_times}")

    # Score trained model.
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    print("tt",save_dir+'/'+model_run)
    
    #save standard model 
    model.save(save_dir+'/'+model_path)
    
    #saving data 
    with open(model_run+'.pickle', 'wb') as handle:
        pickle.dump([training_time,epoch_times,scores[0],scores[1]], handle)

    #save pruned model
    model_for_export = tfmot.sparsity.keras.strip_pruning(model)
    pruned_keras_file = save_dir+'/PRUNE_'+model_path
    keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
    print('Saved pruned Keras model to:', pruned_keras_file)
    
    #convert to tflite+ save
    pruned_tflite_file=save_dir+'/lite_PRUNE_'+model_path
    tflite_conv(model,pruned_tflite_file)
    
    #save to tf lite + qaunt
    pruned_tflite_quant_file=save_dir+'/lite_quant_PRUNE_'+model_path
    tflite_conv(model,pruned_tflite_quant_file,quant=True)
    
    pruned=get_gzipped_model_size(pruned_keras_file)
    tflite_pruned=get_gzipped_model_size(pruned_tflite_file)
    tflite_quant_pruned=get_gzipped_model_size(pruned_tflite_quant_file)
    
    
    print("Size of gzipped pruned Keras model: %.2f bytes" % (pruned))
    print("Size of gzipped pruned TFlite model: %.2f bytes" % (tflite_pruned))
    print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (tflite_quant_pruned))
    
    with open(model_run+'.pickle', 'wb') as handle:
        pickle.dump([training_time,epoch_times,scores[0],scores[1],pruned,tflite_pruned,tflite_quant_pruned], handle)

    return(model,model_path)


## Data: loading and preparation

In [ ]:

# Load the CIFAR10 data.
(X_train, Y_train), (X_test, Y_test) = cifar100.load_data()
# Normalize data.
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# Convert class vectors to binary class matrices.
Y_train = keras.utils.to_categorical(Y_train)
Y_test = keras.utils.to_categorical(Y_test)


## Experimenting with final sparsity hyperparameter



### Resnet44 Experiments with constant Sparsity on V100

In [ ]:
sparsity_level=[0.6,0.8,0.9]
names=["sixty","eighty","ninety"]
models=[]
#for x in sparsity_level:
for x in range(3):
    name="resnet44_const_sparsity_"+names[x]
    print(name)
    model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=7,frequency=100,\
                    initial_sparsity = False, final_sparsity=sparsity_level[x],gpu="V100",\
                    const=True, poly=False,file_name=name,\
                    num_classes=100,begin_step=0,end_step='default')
    models=models+[model1]
    

resnet44_const_sparsity_sixty
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)
Learning rate:  0.001
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv2d_90   (None, 32, 32, 16)  882         ['input_3[0][0]']                
 (PruneLowMagnitude)                                                                              
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d_90[0
 alization_86 (PruneLowMagnitud       

 prune_low_magnitude_add_44 (Pr  (None, 32, 32, 16)  1           ['prune_low_magnitude_activation_
 uneLowMagnitude)                                                90[0][0]',                       
                                                                  'prune_low_magnitude_batch_norma
                                                                 lization_92[0][0]']              
                                                                                                  
 prune_low_magnitude_activation  (None, 32, 32, 16)  1           ['prune_low_magnitude_add_44[0][0
 _92 (PruneLowMagnitude)                                         ]']                              
                                                                                                  
 prune_low_magnitude_conv2d_97   (None, 32, 32, 16)  4626        ['prune_low_magnitude_activation_
 (PruneLowMagnitude)                                             92[0][0]']                       
          

 prune_low_magnitude_conv2d_103  (None, 32, 32, 16)  4626        ['prune_low_magnitude_activation_
  (PruneLowMagnitude)                                            98[0][0]']                       
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d_103[
 alization_99 (PruneLowMagnitud                                  0][0]']                          
 e)                                                                                               
                                                                                                  
 prune_low_magnitude_activation  (None, 32, 32, 16)  1           ['prune_low_magnitude_batch_norma
 _99 (PruneLowMagnitude)                                         lization_99[0][0]']              
                                                                                                  
 prune_low

 de)                                                                                              
                                                                                                  
 prune_low_magnitude_activation  (None, 16, 16, 32)  1           ['prune_low_magnitude_batch_norma
 _105 (PruneLowMagnitude)                                        lization_105[0][0]']             
                                                                                                  
 prune_low_magnitude_conv2d_111  (None, 16, 16, 32)  18466       ['prune_low_magnitude_activation_
  (PruneLowMagnitude)                                            105[0][0]']                      
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 16, 16, 32)  129         ['prune_low_magnitude_conv2d_111[
 alization_106 (PruneLowMagnitu                                  0][0]']                          
 de)      

/opt/conda/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


...

391/391 [==============================] - 38s 97ms/step - loss: 0.8718 - accuracy: 0.8121 - val_loss: 3.4683 - val_accuracy: 0.3508 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 202/300
391/391 [==============================] - ETA: 0s - loss: 0.8709 - accuracy: 0.8149WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 36s 93ms/step - loss: 0.8709 - accuracy: 0.8149 - val_loss: 3.4660 - val_accuracy: 0.3518 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 203/300
391/391 [==============================] - ETA: 0s - loss: 0.8736 - accuracy: 0.8143WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 38s 96ms/step - loss: 0.8736 - accuracy: 0.8143 - val_loss: 3.4668 - val_accuracy: 0.3519 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 204/300
391/391 [==============================] - ETA: 0s - loss: 0.8707 - accuracy: 0.8131WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 37s 94ms/step - loss: 0.8707 - accuracy: 0.8131 - val_loss: 3.4677 - val_accuracy: 0.3505 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 205/300
391/391 [==============================] - ETA: 0s - loss: 0.8663 - accuracy: 0.8154WARNING:tensorflow:Can save best model only with val_acc available, skipping.


391/391 [==============================] - 38s 96ms/step - loss: 0.8663 - accuracy: 0.8154 - val_loss: 3.4662 - val_accuracy: 0.3512 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 206/300
118/391 [========>.....................] - ETA: 24s - loss: 0.8642 - accuracy: 0.8156

### Resnet44 Experiments with Poly Sparsity on V100

In [ ]:
sparsity_level=[0.6,0.8,0.9]

names=["sixty","eighty","ninety"]
models=[]
#for x in sparsity_level:
for x in range(2, 3):
    name="resnet44_poly_sparsity_"+names[x]
    print(name)
    model1 = resnet_training(X_train,Y_train,X_test, Y_test,layers=7,frequency=100,\
                    initial_sparsity = 0.5, final_sparsity=sparsity_level[x],gpu="V100",\
                    const=False, poly=True,file_name=name,\
                    num_classes=100,begin_step=0,end_step='default')
    models=models+[model1]
    

resnet44_poly_sparsity_ninety
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 100)
Learning rate:  0.001
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv2d_45   (None, 32, 32, 16)  882         ['input_2[0][0]']                
 (PruneLowMagnitude)                                                                              
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d_45[0
 alization_43 (PruneLowMagnitud       

 prune_low_magnitude_add_23 (Pr  (None, 32, 32, 16)  1           ['prune_low_magnitude_activation_
 uneLowMagnitude)                                                47[0][0]',                       
                                                                  'prune_low_magnitude_batch_norma
                                                                 lization_49[0][0]']              
                                                                                                  
 prune_low_magnitude_activation  (None, 32, 32, 16)  1           ['prune_low_magnitude_add_23[0][0
 _49 (PruneLowMagnitude)                                         ]']                              
                                                                                                  
 prune_low_magnitude_conv2d_52   (None, 32, 32, 16)  4626        ['prune_low_magnitude_activation_
 (PruneLowMagnitude)                                             49[0][0]']                       
          

 prune_low_magnitude_conv2d_58   (None, 32, 32, 16)  4626        ['prune_low_magnitude_activation_
 (PruneLowMagnitude)                                             55[0][0]']                       
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 32, 32, 16)  65          ['prune_low_magnitude_conv2d_58[0
 alization_56 (PruneLowMagnitud                                  ][0]']                           
 e)                                                                                               
                                                                                                  
 prune_low_magnitude_activation  (None, 32, 32, 16)  1           ['prune_low_magnitude_batch_norma
 _56 (PruneLowMagnitude)                                         lization_56[0][0]']              
                                                                                                  
 prune_low

 e)                                                                                               
                                                                                                  
 prune_low_magnitude_activation  (None, 16, 16, 32)  1           ['prune_low_magnitude_batch_norma
 _62 (PruneLowMagnitude)                                         lization_62[0][0]']              
                                                                                                  
 prune_low_magnitude_conv2d_66   (None, 16, 16, 32)  18466       ['prune_low_magnitude_activation_
 (PruneLowMagnitude)                                             62[0][0]']                       
                                                                                                  
 prune_low_magnitude_batch_norm  (None, 16, 16, 32)  129         ['prune_low_magnitude_conv2d_66[0
 alization_63 (PruneLowMagnitud                                  ][0]']                           
 e)       

 prune_low_magnitude_batch_norm  (None, 16, 16, 32)  129         ['prune_low_magnitude_conv2d_72[0
 alization_69 (PruneLowMagnitud                                  ][0]']                           
 e)                                                                                               
                                                                                                  
 prune_low_magnitude_add_33 (Pr  (None, 16, 16, 32)  1           ['prune_low_magnitude_activation_
 uneLowMagnitude)                                                67[0][0]',                       
                                                                  'prune_low_magnitude_batch_norma
                                                                 lization_69[0][0]']              
                                                                                                  
 prune_low_magnitude_activation  (None, 16, 16, 32)  1           ['prune_low_magnitude_add_33[0][0
 _69 (Prun

...

INFO:tensorflow:Assets written to: /tmp/tmpkc9jgh6v/assets
2022-12-13 02:27:03.318665: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-13 02:27:03.318723: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-13 02:27:03.320047: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpkc9jgh6v
2022-12-13 02:27:03.605914: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-13 02:27:03.605980: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpkc9jgh6v
2022-12-13 02:27:04.464431: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-12-13 02:27:04.613220: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-13 02:27:06.288957: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

Saved pruned TFLite model to: /home/yt2776/COMS6998/final_proj/saved_models/lite_PRUNE_resnet44_poly_sparsity_ninety_44_V100.h5


INFO:tensorflow:Assets written to: /tmp/tmp3qpy1086/assets


INFO:tensorflow:Assets written to: /tmp/tmp3qpy1086/assets
2022-12-13 02:29:14.119487: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-13 02:29:14.119542: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-13 02:29:14.119760: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp3qpy1086
2022-12-13 02:29:14.255486: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-13 02:29:14.255544: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp3qpy1086
2022-12-13 02:29:14.874605: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-13 02:29:16.589607: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp3qpy1086
2022-12-13 02:29:17.228600: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/yt2776/COMS6998/final_proj/saved_models/lite_quant_PRUNE_resnet44_poly_sparsity_ninety_44_V100.h5
Size of gzipped pruned Keras model: 572969.00 bytes
Size of gzipped pruned TFlite model: 634683.00 bytes
Size of gzipped pruned and quantized TFlite model: 634715.00 bytes
